### EDA YELP DATASET BUSINESS

In [1]:
import pandas as pd 
import os 

In [2]:
# Conectamos con el google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Dependencies and libraries

In [ ]:
# Instalamos las dependencias de nuestro entorno de trabajo 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.osuosl.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz
!tar xf spark-3.1.3-bin-hadoop2.7.tgz
!pip install -q findspark
! pip install koalas

In [ ]:
!ls -a

.   .config  sample_data		spark-3.1.3-bin-hadoop2.7.tgz
..  drive    spark-3.1.3-bin-hadoop2.7


In [4]:
# definir variables dle ambiente de linux para conectarse con Java y Spark 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [5]:
import databricks.koalas as ks
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Import datasets 
review = spark.read.load("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/review.json", format="json")
user = spark.read.load("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/user.json", format="json")
business = spark.read.load("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/business.json", format="json")
chechin = spark.read.load("/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/checkin.json", format="json")
tip= spark.read.load('/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/tip.json', format="json")

In [ ]:
# Convert DataFrame from spark to koala
k_review = review.to_koalas() 
k_user = user.to_koalas() 
k_chechin = chechin.to_koalas() 
k_tip = tip.to_koalas() 
k_busines = business.to_koalas() 

In [ ]:
k_user.shape

(1987897, 22)

In [ ]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
# We explore null values per column 
business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               0
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

In [ ]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [ ]:
business[business.address == '']

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
39,fSCNwMtNNQY9QT69Cj9fiA,Sierra Pro Events,,Sparks,NV,89431,39.540154,-119.748395,5.0,7,1,"{'BusinessAcceptsCreditCards': 'True', 'WiFi':...","Musicians, DJs, Karaoke, Event Planning & Serv...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
51,REwfwz-_-CMQ7Np5UVi9Qg,Jennie Deckert,,Carmel,IN,46032,39.969300,-86.165002,5.0,7,1,None,"Real Estate Agents, Home Services, Real Estate",None
66,n7AQvGvNHlmun3kqXeBKVQ,Roy's Appliance Service,,Meridian,ID,83646,43.643494,-116.436000,5.0,5,1,{'BusinessAcceptsCreditCards': 'True'},"Local Services, Appliances & Repair","{'Monday': '0:0-0:0', 'Tuesday': '8:30-15:0', ..."
72,7PDi_iyik3jraDAzWwwR4Q,Chase JP Morgan Bank Credit Card Services,,Wilmington,DE,19850,39.749361,-75.643331,1.5,111,1,None,"Banks & Credit Unions, Financial Services",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150196,CWaEYOc5g4AB-h88lnKntQ,Driving Miss Daisy Shuttle,,Nashville,TN,37214,36.140540,-86.672641,3.0,6,0,None,"Taxis, Transportation, Hotels & Travel",None
150230,IRBhPAC4ZoDpXazpoB3epQ,Good Stuff Baked Treats,,Santa Barbara,CA,93101,34.420334,-119.710749,5.0,9,1,"{'BikeParking': 'False', 'RestaurantsPriceRang...","Food, Food Delivery Services, Bakeries, Desser...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150286,Vl0oo3jjuGpgMWaCbN5r9w,Steve Bright Handyman,,Blue Bell,PA,19422,40.165393,-75.282905,4.5,6,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Cabinetry, Home Services, Home Automation, Han...","{'Monday': '9:0-18:0', 'Tuesday': '9:0-17:0', ..."
150303,JhSByBTYY1rGstRy76YmLA,Reiki with Darren,,Santa Barbara,CA,93105,34.420831,-119.698190,5.0,7,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Health & Medical, Supernatural Readings, Hospi...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."


In [ ]:
def unique_values(data): 
  '''
  > This function detects the unique values in the dataframe
  :: param data: DataFrame of koalas

  Return: 
  ---------
  Return a list with unique values for each column of the DataFrame
  '''
  dict_val_unicos = {}

  columnas = data.columns 
  for columna in columnas: 
    valores_unicos = data[columna].unique()
    dict_val_unicos[columna] = valores_unicos

  return dict_val_unicos

En la columna state borramos la fila de XMS por tener datos que no coinciden


In [ ]:
business.attributes.value_counts()

In [ ]:
business.categories#.nunique()

In [ ]:
business.hours.value_counts()

ATTRIBUTES

In [ ]:
business.attributes

0                             {'ByAppointmentOnly': 'True'}
1                    {'BusinessAcceptsCreditCards': 'True'}
2         {'BikeParking': 'True', 'BusinessAcceptsCredit...
3         {'RestaurantsDelivery': 'False', 'OutdoorSeati...
4         {'BusinessAcceptsCreditCards': 'True', 'Wheelc...
                                ...                        
150341    {'ByAppointmentOnly': 'False', 'RestaurantsPri...
150342    {'BusinessAcceptsCreditCards': 'True', 'Restau...
150343    {'RestaurantsPriceRange2': '1', 'BusinessAccep...
150344    {'BusinessParking': '{'garage': False, 'street...
150345    {'WheelchairAccessible': 'True', 'BusinessAcce...
Name: attributes, Length: 150346, dtype: object

In [8]:
attributes = pd.json_normalize(business['attributes'])

In [9]:
attributes.set_index(business.index, inplace=True)
attributes['business_id'] = attributes.index
attributes.reset_index(drop=True, inplace=True)

In [ ]:
attributes

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,business_id
0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,False,False,True,1,NaN,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,True,True,NaN,NaN,True,NaN,False,NaN,"{'garage': None, 'street': None, 'validated': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150341
150342,NaN,True,True,2,NaN,NaN,NaN,NaN,u'no',"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150342
150343,NaN,True,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150343
150344,NaN,True,True,4,NaN,None,None,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150344


In [ ]:
attributes.ByAppointmentOnly.value_counts()

False    26690
True     15609
None        40
Name: ByAppointmentOnly, dtype: int64

In [ ]:
attributes.RestaurantsAttire.value_counts()

u'casual'    23190
'casual'     15154
u'dressy'      530
'dressy'       273
'formal'        38
None            38
u'formal'       32
Name: RestaurantsAttire, dtype: int64

In [ ]:
attributes.isnull().sum()

ByAppointmentOnly             108007
BusinessAcceptsCreditCards     30581
BikeParking                    77708
RestaurantsPriceRange2         65032
CoatCheck                     144762
RestaurantsTakeOut             90489
RestaurantsDelivery            94064
Caters                        110219
WiFi                           93432
BusinessParking                59261
WheelchairAccessible          121393
HappyHour                     135175
OutdoorSeating                101544
HasTV                         105262
RestaurantsReservations       105099
DogsAllowed                   132062
Alcohol                       107157
GoodForKids                    96971
RestaurantsAttire             111091
Ambience                      106067
RestaurantsTableService       130364
RestaurantsGoodForGroups      106176
DriveThru                     142586
NoiseLevel                    112353
GoodForMeal                   121259
BusinessAcceptsBitcoin        132916
Smoking                       145779
M

In [10]:
attributes.loc[attributes.BusinessParking == 'None']="{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"
attributes.BusinessParking = attributes.BusinessParking.fillna("{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}") # a los valore nulos le pogo False


In [11]:
attributes.loc[attributes.Ambience == 'None']="{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}"
attributes.Ambience = attributes.Ambience.fillna("{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False}")

In [ ]:
attributes.Ambience[20]

"{'romantic': False, 'intimate': False, 'classy': False, 'hipster': True, 'touristy': False, 'trendy': True, 'upscale': False, 'casual': False}"

In [46]:
attributes.isnull().sum()

ByAppointmentOnly             106275
BusinessAcceptsCreditCards     30390
BikeParking                    77008
RestaurantsPriceRange2         64560
CoatCheck                     142250
RestaurantsTakeOut             89149
RestaurantsDelivery            92694
Caters                        108460
WiFi                           91948
BusinessParking                    0
WheelchairAccessible          119690
HappyHour                     132879
OutdoorSeating                 99928
HasTV                         103614
RestaurantsReservations       103497
DogsAllowed                   129790
Alcohol                       105404
GoodForKids                    95510
RestaurantsAttire             109307
Ambience                           0
RestaurantsTableService       128145
RestaurantsGoodForGroups      104495
DriveThru                     140295
NoiseLevel                    110417
GoodForMeal                   119219
BusinessAcceptsBitcoin        130596
Smoking                       143240
M

In [12]:
import json
import ast

In [13]:
def dicc(row):
  #result = json.loads(row)
  result = ast.literal_eval(row)
  return result


In [14]:
BusinessParking = pd.io.json.json_normalize(attributes.BusinessParking.apply(dicc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [15]:
Ambience = pd.io.json.json_normalize(attributes.Ambience.apply(dicc))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [16]:
BusinessParking.set_index(business.index, inplace=True)

In [ ]:
BusinessParking.isnull().sum()

garage         2246
street         3648
validated      2475
lot            2615
valet           350
romantic     150015
intimate     150015
classy       150015
hipster      150015
touristy     150015
trendy       150015
upscale      150015
casual       150015
dtype: int64

In [ ]:
Ambience.isnull().sum()

romantic       5561
intimate       6453
classy         5815
hipster        6618
touristy       5861
trendy         7537
upscale        5146
casual         4453
garage       148075
street       148075
validated    148075
lot          148075
valet        148075
divey        112440
dtype: int64

In [ ]:
attributes.shape

(150346, 40)

hour

In [17]:
hour = pd.read_csv('/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/csv/openhours.csv')

In [ ]:
hour

,business_id,MondayOpen,MondayClose,TuesdayOpen,TuesdayClose,WednesdayOpen,WednesdayClose,ThursdayOpen,ThursdayClose,FridayOpen,FridayClose,SaturdayOpen,SaturdayClose,SundayOpen,SundayClose
0,Pns2l4eNsfO8kk83dixA6A,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
1,mpf3x-BjTdTEA3yCZrAYPw,0:0,0:0,8:0,18:30,8:0,18:30,8:0,18:30,8:0,18:30,8:0,14:0,0:0,0:0
2,tUFrWirKiKi_TAnsVWINQQ,8:0,22:0,8:0,22:0,8:0,22:0,8:0,22:0,8:0,23:0,8:0,23:0,8:0,22:0
3,MTSW4McQd7CbVtyjqoe9mw,7:0,20:0,7:0,20:0,7:0,20:0,7:0,20:0,7:0,21:0,7:0,21:0,7:0,21:0
4,mWMc6_wTdE0EUBKIGXDVfA,0:0,0:0,0:0,0:0,14:0,22:0,16:0,22:0,12:0,22:0,12:0,22:0,12:0,18:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,10:0,19:30,10:0,19:30,10:0,19:30,10:0,19:30,10:0,19:30,10:0,17:30,11:0,17:0
150342,c8GjPIOTGVmIemT7j5_SyQ,9:30,17:30,9:30,17:30,9:30,17:30,9:30,17:30,9:30,17:30,9:0,17:0,0:0,16:0
150343,_QAMST-NrQobXduilWEqSw,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0,0:0
150344,mtGm22y5c2UHNXDFAjaPNw,9:0,20:0,9:0,20:0,9:0,20:0,9:0,20:0,9:0,20:0,9:0,17:0,10:0,17:0


In [18]:
hour = hour.fillna('0:0-0:0') # a los valore nulos le pogo 0:0-0:0
hour

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,business_id
0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,Pns2l4eNsfO8kk83dixA6A
1,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,0:0-0:0,mpf3x-BjTdTEA3yCZrAYPw
2,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0,tUFrWirKiKi_TAnsVWINQQ
3,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0,MTSW4McQd7CbVtyjqoe9mw
4,0:0-0:0,0:0-0:0,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0,mWMc6_wTdE0EUBKIGXDVfA
...,...,...,...,...,...,...,...,...
150341,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-17:30,11:0-17:0,IUQopTMmYQG-qRtBk-8QnA
150342,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:0-17:0,0:0-16:0,c8GjPIOTGVmIemT7j5_SyQ
150343,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,_QAMST-NrQobXduilWEqSw
150344,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-17:0,10:0-17:0,mtGm22y5c2UHNXDFAjaPNw


In [19]:
def funcionOpen(row):
    row=row.replace('-','  ')
    return row.split()[0]


In [20]:
def funcionClose(row):
    row= row.replace('-','  ')
    return row.split()[1]

In [21]:
funcionClose(hour.Monday[2])

'22:0'

In [22]:
hour['MondayOpen'] = hour['Monday'].apply(funcionOpen)

In [23]:
hour['MondayClose'] = hour['Monday'].apply(funcionClose)

In [ ]:
hour.columns

Index(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday', 'business_id', 'MondayOpen', 'MondayClose'],
      dtype='object')

In [24]:
hour['TuesdayOpen']= hour['Tuesday'].apply(funcionOpen)
hour['TuesdayClose']= hour['Tuesday'].apply(funcionClose)

In [25]:
hour['WednesdayOpen']= hour['Wednesday'].apply(funcionOpen)
hour['WednesdayClose']= hour['Wednesday'].apply(funcionClose)
hour['ThursdayOpen']= hour['Thursday'].apply(funcionOpen)
hour['ThursdayClose']= hour['Thursday'].apply(funcionClose)

In [26]:
hour['FridayOpen']= hour['Friday'].apply(funcionOpen)
hour['FridayClose']= hour['Friday'].apply(funcionClose)
hour['SaturdayOpen']= hour['Saturday'].apply(funcionOpen)
hour['SaturdayClose']= hour['Saturday'].apply(funcionClose)

In [27]:
hour['SundayOpen']= hour['Sunday'].apply(funcionOpen)
hour['SundayClose']= hour['Sunday'].apply(funcionClose)

In [28]:
hour.drop(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday'], axis=1,inplace=True)

In [ ]:
hour.describe()

In [72]:
hour.SundayOpen.value_counts()

0:0      76398
11:0     18163
10:0     11031
12:0     10240
9:0       7096
         ...  
1:45         1
0:15         1
16:15        1
6:29         1
17:15        1
Name: SundayOpen, Length: 78, dtype: int64

In [73]:
hour.SundayClose.value_counts()

0:0      79297
21:0     10613
22:0      9776
17:0      9281
18:0      8373
         ...  
19:40        1
6:15         1
5:15         1
23:55        1
1:59         1
Name: SundayClose, Length: 93, dtype: int64

In [ ]:
hour.isnull().sum()

business_id       0
MondayOpen        0
MondayClose       0
TuesdayOpen       0
TuesdayClose      0
WednesdayOpen     0
WednesdayClose    0
ThursdayOpen      0
ThursdayClose     0
FridayOpen        0
FridayClose       0
SaturdayOpen      0
SaturdayClose     0
SundayOpen        0
SundayClose       0
dtype: int64

cetegories


In [ ]:
categories = pd.read_csv('/content/drive/MyDrive/HENRY_TRABAJO_GRUPAL/Repository/Data/csv/categories.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
categories

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,Doctors,Traditional Chinese Medicine,Naturopathic/Holistic,Acupuncture,Health & Medical,Nutritionists,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Shipping Centers,Local Services,Notaries,Mailbox Centers,Printing Services,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Department Stores,Shopping,Fashion,Home & Garden,Electronics,Furniture Stores,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Restaurants,Food,Bubble Tea,Coffee & Tea,Bakeries,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brewpubs,Breweries,Food,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,Nail Salons,Beauty & Spas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,Pets,Nurseries & Gardening,Pet Stores,Hobby Shops,Bird Shops,Home & Garden,Shopping,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,Shopping,Jewelry,Piercing,Toy Stores,Beauty & Spas,Accessories,Fashion,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,Fitness/Exercise Equipment,Eyewear & Opticians,Shopping,Sporting Goods,Bikes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


o = tipo negocio
1 = servicio que brinda
2 a 35 = productos

In [ ]:
categories.rename(columns = {'0':'Business_type','1':'services','2':'product'}, inplace=True) 

In [ ]:
categories.columns

Index(['Business_type', 'services', 'product', '3', '4', '5', '6', '7', '8',
       '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32',
       '33', '34', '35'],
      dtype='object')

In [ ]:
categories.services.value_counts()

 Restaurants           14126
 Food                   6312
 Shopping               5350
 Beauty & Spas          4152
 Home Services          3746
                       ...  
 Indoor Landscaping        1
 Childproofing             1
 Meaderies                 1
 LAN Centers               1
 Carpet Dyeing             1
Name: services, Length: 1165, dtype: int64

In [ ]:
categories.Business_type.value_counts()

Restaurants                      15290
Food                              6783
Shopping                          5480
Beauty & Spas                     4385
Home Services                     3793
                                 ...  
Squash                               1
Tuscan                               1
Audio/Visual Equipment Rental        1
Sports Betting                       1
Mortuary Services                    1
Name: Business_type, Length: 1159, dtype: int64

In [ ]:
categories.isnull().sum()

0        103
1        484
2      33047
3      60112
4      86466
5     109916
6     126149
7     136338
8     142207
9     145805
10    147829
11    148912
12    149559
13    149932
14    150110
15    150198
16    150258
17    150299
18    150320
19    150330
20    150334
21    150337
22    150340
23    150342
24    150343
25    150344
26    150345
27    150345
28    150345
29    150345
30    150345
31    150345
32    150345
33    150345
34    150345
35    150345
dtype: int64